# Basic Example

In [10]:
# !pip install gymnasium
# !pip install ray==2.7.0
# !pip install -U ray[rllib]==2.7.0
# !pip install torch
import csbsingleenv
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb
import wandb

In [11]:
wandb.init(entity="rl-class-2024",
           project="RL-Projects")

In [8]:
setup_wandb(config={"entity": "rl-class-2024",
                    "project": "RL-Projects"})

2024-05-09 16:12:37,084	INFO wandb.py:308 -- Already logged into W&B.


Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [12]:
env = csbsingleenv.CodersStrikeBackSingle()

In [13]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    rewards.append(reward)
print(sum(rewards))

-10100


In [14]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [15]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-05-09 16:14:50,206	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.7.0


In [16]:
config = (PPOConfig()
          .environment(csbsingleenv.CodersStrikeBackSingle)
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")]),
)

tuner.fit()

2024-05-09 16:14:50,939	INFO wandb.py:308 -- Already logged into W&B.
2024-05-09 16:14:50,954	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=341560) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=341560) 2024-05-09 16:14:54,184	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defaul

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.196190423103792}, 'default_policy': {'total_loss': 7.196190423103792, 'policy_loss': -0.05820031542388567, 'vf_loss': 7.242022033502806, 'vf_loss_unclipped': 5108058.236227343, 'vf_explained_var': -0.0013265527133494298, 'entropy': 4.3874739146690125, 'mean_kl_loss': 0.027486110997128377, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10195.0, 'episode_reward_mean': -10103.1, 'episode_len_mean': 103.1, 'episode_media': {}, 'episodes_this_iter': 39, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean

# Empty Customized Environment

In [17]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

In [18]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [19]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(num_cpus=4, runtime_env=runtime_env)

Failed to finish cpu monitoring: cannot join thread before it is started
2024-05-09 16:15:35,316	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-09 16:15:35,500	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-09 16:15:35,690	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_549991fb1282f84a.zip' (9.61MiB) to Ray cluster...
2024-05-09 16:15:35,731	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_549991fb1282f84a.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [20]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle()})
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")]),
)

tuner.fit()

2024-05-09 16:15:36,484	INFO wandb.py:308 -- Already logged into W&B.
2024-05-09 16:15:36,497	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=343049) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=343049) 2024-05-09 16:15:40,674	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defaul

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.0571148387913}, 'default_policy': {'total_loss': 7.0571148387913, 'policy_loss': -0.04674354745253825, 'vf_loss': 7.090280552408588, 'vf_loss_unclipped': 4859357.904144612, 'vf_explained_var': 4.804210622173383e-06, 'entropy': 5.0039524554189585, 'mean_kl_loss': 0.03017300255010838, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10145.0, 'episode_reward_mean': -10103.13, 'episode_len_mean': 103.13, 'episode_media': {}, 'episodes_this_iter': 37, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {

# Real Customized Environment

In [21]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

In [22]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces, logger

# Note, this is only meant as an example of how to write a custom environment.
# I doubt it'll actually learn well.
class CustomizedRacing:
    def __init__(self):
        min_pos = -200000.0
        max_pos = 200000.0
        min_vel = -2000.0
        max_vel = 2000.0
        screen_max = [16000,9000]
        # This observation space will only have the next checkpoint rather than the next 5.
        self.observation_space = spaces.Box(
            low=np.array([0, -np.pi, min_pos, min_pos, min_vel, min_vel]+[0,0]),
            high=np.array([3, np.pi, max_pos, max_pos, max_vel, max_vel]+screen_max),
            dtype=np.float64
        )

        # This action will be how much to turn, rather than a point to turn towards.
        self.action_space = spaces.Box(
            low = np.array([-1.0, 0.0]),
            high = np.array([1.0, 100]),
            dtype=np.float64
        )

    # We can define and reset any extra variables we want to keep track of here.
    def reset(self):
        self.last_checkpoint = None
        self.last_sqr_dist = None

    def reward(self, env, rew):
        # Add reward for getting closer to the next checkpoint
        cur_dist = csbsingleenv.get_sqr_distance(env.pos, env.next_checkpoint())
        if self.last_sqr_dist is not None and cur_dist < self.last_sqr_dist:
            rew += 500
        self.last_sqr_dist = cur_dist
        
        # Add reward for passing a checkpoint
        cur_checkpoint = env.checkpoint_index
        if self.last_checkpoint is not None and cur_checkpoint != self.last_checkpoint:
            rew += 600
        self.last_checkpoint = cur_checkpoint
        
        if env.failed:
            return -1000
        return rew
    
    # Need to define the traformation.  Should be easy here since we just need to drop the excess data
    def transform_observation(self, env, obs):
        return obs[:8]

    # Need to also transform the angle change into a point to aim at.
    # Will define it as 1000 distance from the angle relative to current pointed direction
    def transform_action(self, env, action):
        preferred_dir = env.theta + action[0]*env.maxSteeringAngle
        dir_vector = 1000*csbsingleenv.angle_to_vector(preferred_dir)
        point_to_aim_at = env.pos + dir_vector
        trans_action = np.array([point_to_aim_at.x, point_to_aim_at.y, action[1]])
        return trans_action
    

In [23]:
env = env_customizer.CustomizedEnvironment(
    env_config={"env":csbsingleenv.CodersStrikeBackSingle(),
                "customizer":CustomizedRacing()})

In [24]:
env.reset()

(array([3.00000000e+00, 7.67588342e-01, 1.51900000e+03, 4.89800000e+03,
        0.00000000e+00, 0.00000000e+00, 5.31900000e+03, 8.56500000e+03]),
 {})

In [25]:
env.step(np.array([0,100]))

(array([3.00000000e+00, 7.67588342e-01, 1.59100000e+03, 4.96700000e+03,
        6.03500000e+01, 5.86500000e+01, 5.31900000e+03, 8.56500000e+03]),
 -1,
 False,
 False,
 {})

In [26]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [27]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(runtime_env=runtime_env)

Failed to finish cpu monitoring: cannot join thread before it is started
Failed to finish cpu monitoring: cannot join thread before it is started
Failed to finish cpu monitoring: cannot join thread before it is started
Failed to finish cpu monitoring: cannot join thread before it is started
Failed to finish cpu monitoring: cannot join thread before it is started
Failed to finish cpu monitoring: cannot join thread before it is started
2024-05-09 16:16:23,801	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-09 16:16:23,957	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-09 16:16:24,149	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_549991fb1282f84a.zip' (9.61MiB) to Ray cluster...
2024-05-09 16:16:24,210	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_549991fb1282f84a.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [28]:
checkpoint_config = air.CheckpointConfig(checkpoint_frequency=5)

In [29]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle(),
                                 "customizer" : CustomizedRacing()
                                 })
          .framework('torch')
          .training(gamma=0.99,
                    model= {
                              "use_lstm": True,
                              "lstm_cell_size": 64,
                              "max_seq_len": 10
                           })
          .rollouts(num_rollout_workers=20, num_envs_per_worker=5, rollout_fragment_length="auto")
          .resources(num_gpus=4, num_learner_workers=4, num_gpus_per_learner_worker=1)
)
config.num_sgd_iter = 6
stop = {"timesteps_total": 3000000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop,
                             storage_path="~/Desktop/Tristan/rl-projects/results",
                             checkpoint_config=checkpoint_config,
                             callbacks=[WandbLoggerCallback(project="RL-Projects",upload_checkpoints=True,
                                                            save_checkpoints=True, group="rl-class-2024")],)

)

tuner.fit()


2024-05-09 16:16:25,494	INFO wandb.py:308 -- Already logged into W&B.
2024-05-09 16:16:25,508	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=344967) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=344967) 2024-05-09 16:16:29,552	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a defaul